In [4]:
if (!require(RJDBC)) { install.packages(RJDBC); require('RJDBC') }
if (!require(rjson)) { install.packages(rjson); require('rjson') }

Loading required package: rjson



# Get the connector to dremio with the class DremioQueryMakerJDBC

In [5]:
# Credencials para a conexão no Dremio
credentials <- fromJSON(file="/home/ubuntu/notebooks/files/.credentials.json")

cnct_str = paste("jdbc:dremio:direct=", credentials['host'], ":", credentials['port'], sep = "")
drv = JDBC("com.dremio.jdbc.Driver",
           "/home/ubuntu/notebooks/files/dremio-jdbc-driver-4.1.3-202001022113020736-53142377.jar")
conn = dbConnect(drv, cnct_str, credentials['uid'], credentials["pwd"])

index_query = "SELECT * FROM INFORMATION_SCHEMA.VIEWS"
index = dbGetQuery(conn, index_query)
index

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,VIEW_DEFINITION
,<chr>,<chr>,<chr>,<chr>
1,DREMIO,covidlake-raw,brasil_io,"SELECT * FROM ""brasil.io"""
2,DREMIO,covidlake-raw,google_trends,"SELECT * FROM ""google_trends.parquet"""
3,DREMIO,covidlake-raw,health_ministry,SELECT * FROM health_ministry
4,DREMIO,covidlake-raw,hospitals_dimension,"SELECT * FROM ""hospitals.csv"""
5,DREMIO,covidlake-raw,wcota,SELECT * FROM wcota
6,DREMIO,PowerBI,VDS-cases-state,"select a.state,a.city, sum(a.confirmed) as cases, ""a"".""date"" from ""covidlake-raw"".""brasil_io"" as a where a.is_last = true group by a.state,a.city,""a"".""date"" order by 4,3 desc"
7,DREMIO,covidlake-raw.ipea,dolar_ratio,"SELECT * FROM ""dolar_ratio.parquet"""


# Get tables by index number

In [6]:
index_number = 2
table_name = index$TABLE_NAME[index_number]
source = index$TABLE_SCHEMA[index_number]
query = paste("SELECT * FROM \"", source, "\".\"", table_name, "\"", sep = "")
table = dbGetQuery(conn, query)
head(table)

,index,TIME,Medicamento_VALUE,Vacina_VALUE,Sintoma_VALUE,Respirador_VALUE
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0,2004-01-01,72,35,10,13
2,1,2004-02-01,87,NA,12,11
3,2,2004-03-01,84,NA,11,19
4,3,2004-04-01,80,NA,13,8
5,4,2004-05-01,93,NA,13,7
6,5,2004-06-01,77,NA,13,8


# Make Free Queries

In [ ]:
query = '
SELECT estado, count(*) AS "count"
FROM "covidlake-raw"."health_ministry"
GROUP BY estado
;'
table = dbGetQuery(conn, query)
head(table)